In [1]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 34.5 MB/s eta 0:00:00
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 4.39 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2025-11-13 18:03:50--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.142.26, 104.192.142.25, 104.192.142.24, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.26|:443... connected.
HTTP request sent, 

In [9]:
# 라이브러리 싹 다 모아봤습니다.
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import konlpy

# ⭐⭐⭐ [NEW] MECAB 임포트 추가 ⭐⭐⭐
from konlpy.tag import Mecab
from tqdm import tqdm
from collections import Counter
# ⭐⭐⭐ [END NEW] ⭐⭐⭐

print(torch.__version__)
print(np.__version__)
print(pd.__version__)
print(matplotlib.__version__)
print(konlpy.__version__)


2.8.0+cu126
2.0.2
2.2.2
3.10.0
0.6.0


데이터 업로드!

In [10]:
train = pd.read_table("/content/ratings_train.txt")
test = pd.read_table("/content/ratings_test.txt")

print(train.info())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
None


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


수동으로 데이터 업로딩 안하면 안됩니다!! ㅜ 다음 블럭 시작전에.. 나는야 코랩 유저..

In [14]:

# ================================================================================
# ✅ 셀 2: 데이터 로드 (기존 코드 - 변경 없음)
# ================================================================================
train = pd.read_table("/content/ratings_train.txt")
test = pd.read_table("/content/ratings_test.txt")

print(train.info())
train.head()


# ================================================================================
# ✅ 셀 4: 텍스트 전처리 (기존 코드 - 변경 없음)
# ================================================================================
import re

# 불용어 리스트
STOPWORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 맞춤법 변형 사전
SPELLING_DICT = {
    '굳': ['궅', '굳', '굿'],
    '미쳤': ['미첫', '미쳣', '미첬', '미쳤', 'ㅁㅊ'],
    '괜찮': ['괜찮', '괜춘', '괜찬', 'ㄱㅊ', '갠찬', '갠찮', '괸찬', '괸찮'],
    '봤': ['봣'],
    '겠': ['겟']
}

# 텍스트 컬럼명
TEXT_COL = "document"

def preprocess_text(text):
    """
    텍스트 전처리 함수
    """
    if pd.isna(text):
        return ""

    text = str(text)

    # 1. 반복되는 문장 부호 제거 (2개 이상 → 1개)
    text = re.sub(r'([.!?…;])\1+', r'\1', text)

    # 2. 맞춤법 변형 통일
    for correct, variations in SPELLING_DICT.items():
        for variant in variations:
            text = text.replace(variant, correct)

    # 3. 반복 문자 제거 (3번 이상 반복 → 2번)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # 4. 자음/모음 단독 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', text)

    # 5. 영어/숫자/한글/문장부호만 남기고 모두 삭제
    text = re.sub(r'[^가-힣a-zA-Z0-9\s.!?,]', ' ', text)

    # 6. 문장부호 앞뒤로 공백 추가
    text = re.sub(r'([.!?,])', r' \1 ', text)

    # 7. 영어 소문자 변환
    text = text.lower()

    # 8. 불용어 제거 (조사가 붙은 경우도 처리)
    words = text.split()

    # 단어 끝에 불용어가 붙어있으면 제거
    filtered_words = []
    for word in words:
        # 단어 전체가 불용어인 경우
        if word in STOPWORDS:
            continue
        # 단어 끝에서 불용어 제거 (가장 긴 것부터 체크)
        found = False
        for stopword in sorted(STOPWORDS, key=len, reverse=True):
            if len(word) > len(stopword) and word.endswith(stopword):
                cleaned = word[:-len(stopword)]
                if len(cleaned) >= 2:
                    filtered_words.append(cleaned)
                    found = True
                    break
        if not found:
            filtered_words.append(word)

    text = ' '.join(filtered_words)

    # 9. 연속 공백을 하나의 공백으로 교체
    text = re.sub(r'\s+', ' ', text)

    # 10. 앞뒤 공백 제거
    text = text.strip()

    return text


def preprocess_dataframe(df, text_col=TEXT_COL):
    """
    데이터프레임 전처리 함수
    """
    print(f"전처리 전 데이터 크기: {len(df)}")

    # 1. 결측치 제거
    df = df.dropna(subset=[text_col])
    print(f"결측치 제거 후: {len(df)}")

    # 2. 텍스트 전처리 적용
    df[text_col] = df[text_col].apply(preprocess_text)

    # 3. 전처리 후 빈 문자열 제거
    df = df[df[text_col].str.strip() != '']
    print(f"빈 문자열 제거 후: {len(df)}")

    # 4. 중복 행 제거
    df = df.drop_duplicates(subset=[text_col])
    print(f"중복 제거 후: {len(df)}")

    # 인덱스 재설정
    df = df.reset_index(drop=True)

    return df


# 샘플 텍스트로 테스트
sample_texts = [
    "어제 본 영화 진짜 재밌었음!!! 또 보고 싶어 😂",
    "나는 오늘 아침에 학교에 갔다. 근데 너무 졸렸음ㅋㅋㅋㅋ",
    "밥은 먹었니?? 아직이야... 점심에 같이 먹자!!!",
    "메캅 형태소 분석은 한국어 처리에서 많이 사용돼 👍",
    "파이썬으로 토큰 빈도와 품사 분포를 시각화해 보자!!!",
    "요즘 코사인 유사도 기반 벡터 검색으로 RG 구축을 많이 해!!",
    "에이전트는 외부 도구를 호출해 작업을 자동화할 수 있어. 굳!",
    "이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ",
    "배우 연기 굳이 훌륭했음, 스토리는 봣지만...",
    "이건 ㄱㅊ 영화네, 굿굿!"
]

print("=" * 80)
print("샘플 텍스트 전처리 결과")
print("=" * 80)
for i, text in enumerate(sample_texts, 1):
    processed = preprocess_text(text)
    print(f"\n[{i}] 원본: {text}")
    print(f"    결과: {processed}")

print("\n" + "=" * 80)
print("실제 데이터 적용 예시")
print("=" * 80)

train_processed = preprocess_dataframe(train.copy())
test_processed = preprocess_dataframe(test.copy())

print("\n전처리 완료!")
print(f"Train 데이터: {len(train_processed)}개")
print(f"Test 데이터: {len(test_processed)}개")

print("\n전처리 결과 샘플:")
print(train_processed.head(10))


# ================================================================================
# ⭐⭐⭐ [NEW] 셀 4-1: MECAB 형태소 분석 추가 ⭐⭐⭐
# ================================================================================
print("\n" + "=" * 80)
print("🔧 MECAB 형태소 분석기 초기화")
print("=" * 80)

# MECAB 초기화
mecab = Mecab()
print("✅ MECAB 로드 완료")

# MECAB 동작 테스트
test_result = mecab.morphs("이 영화 정말 재미있었어")
print(f"✅ MECAB 정상 작동")
print(f"분석 결과: {test_result}")


def mecab_morphs_analysis(text):
    """
    MECAB 형태소 분석 함수
    - 입력: 전처리된 텍스트
    - 출력: 형태소 리스트 (공백으로 결합)
    """
    if pd.isna(text) or text.strip() == '':
        return ""

    try:
        morphs = mecab.morphs(text)
        return ' '.join(morphs)
    except:
        return text


def apply_mecab_to_dataframe(df, text_col=TEXT_COL):
    """
    데이터프레임에 MECAB 형태소 분석 적용
    """
    print(f"\n🔄 MECAB 형태소 분석 처리 중...")
    df['document_morphs'] = df[text_col].apply(
        lambda x: mecab_morphs_analysis(x) if pd.notna(x) else ""
    )

    # 분석 결과 샘플 표시
    print(f"\n📊 MECAB 분석 결과 샘플:")
    print("=" * 100)
    for i in range(min(3, len(df))):
        print(f"\n[샘플 {i+1}]")
        print(f"  원본:      {df[text_col].iloc[i][:80]}")
        print(f"  형태소:    {df['document_morphs'].iloc[i][:80]}")
    print("=" * 100)

    return df


# Train/Test 데이터에 MECAB 분석 적용
train_processed = apply_mecab_to_dataframe(train_processed)
test_processed = apply_mecab_to_dataframe(test_processed)

print(f"\n✅ MECAB 형태소 분석 완료!")
print(f"   Train: {len(train_processed)}개 문장 분석")
print(f"   Test: {len(test_processed)}개 문장 분석")

# ⭐⭐⭐ [END NEW] ⭐⭐⭐


# ================================================================================
# ✅ 셀 6: SentencePiece 설치 (기존 코드 - 변경 없음)
# ================================================================================
!pip install sentencepiece


# ================================================================================
# ✅ 셀 7: SentencePiece 학습 (수정: MECAB 형태소 분석 결과 사용)
# ================================================================================
import sentencepiece as spm
import os

# 학습용 corpus 파일 생성
corpus_file = 'naver_reviews.train.temp'

# ⭐⭐⭐ [MODIFIED] 기존: train_processed['document'] → 수정: train_processed['document_morphs'] ⭐⭐⭐
with open(corpus_file, 'w', encoding='utf-8') as f:
    for text in train_processed['document_morphs']:  # 변경: MECAB 형태소 분석 결과 사용
        f.write(text + '\n')
# ⭐⭐⭐ [END MODIFIED] ⭐⭐⭐

# SentencePiece 학습
vocab_size = 8000
model_prefix = 'naver_spm'

spm.SentencePieceTrainer.Train(
    f'--input={corpus_file} '
    f'--model_prefix={model_prefix} '
    f'--vocab_size={vocab_size} '
    f'--model_type=bpe '
    f'--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 '
    f'--pad_piece=<PAD> --unk_piece=<UNK> --bos_piece=<BOS> --eos_piece=<EOS>'
)

!ls -l naver_spm*


# ================================================================================
# ✅ 셀 8: SentencePiece 학습된 모델 활용 (수정: MECAB 형태소 분석 결과 사용)
# ================================================================================
# 학습된 SentencePiece 모델 활용 예시
sp = spm.SentencePieceProcessor(model_file=f'{model_prefix}.model')

sample_text = "이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ"

# ⭐⭐⭐ [MODIFIED] MECAB 형태소 분석 결과를 SentencePiece로 인코딩 ⭐⭐⭐
sample_text_preprocessed = preprocess_text(sample_text)
sample_text_morphs = mecab_morphs_analysis(sample_text_preprocessed)  # MECAB 형태소 분석

encoded_ids = sp.encode(sample_text_morphs,  # 변경: MECAB 형태소 분석 결과 사용
                        out_type=int, add_bos=True, add_eos=True)
# ⭐⭐⭐ [END MODIFIED] ⭐⭐⭐

decoded_text = sp.decode(encoded_ids)

print("=" * 80)
print("📊 SentencePiece 토큰화 예시 (MECAB 형태소 분석 결과 기반)")
print("=" * 80)
print(f"원본 텍스트:       {sample_text}")
print(f"정규식 전처리:     {sample_text_preprocessed}")
print(f"MECAB 형태소:      {sample_text_morphs}")
print(f"Encoded IDs:       {encoded_ids}")
print(f"Decoded Text:      {decoded_text}")


# ================================================================================
# ✅ 셀 12: SentencePiece 토큰화 함수 (기존 코드 - 변경 없음)
# ================================================================================
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(torch.tensor(s.EncodeAsIds(sen), dtype=torch.long))

    with open("./naver_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({word:idx})
        index_word.update({idx:word})

    tensor = pad_sequence(tensor, batch_first=True, padding_value=0)

    return tensor, word_index, index_word


# ⭐⭐⭐ [NEW] 토큰화 및 텐서 변환 추가 ⭐⭐⭐
print("\n" + "=" * 80)
print("📊 데이터 토큰화 및 텐서 변환")
print("=" * 80)

# PAD_IDX가 정의되지 않았을 수 있으므로 여기서 다시 정의
PAD_IDX = 0 # 공통 하이퍼파라미터 셀에 있지만, 여기서 사용되므로 함께 정의

# 훈련 데이터 토큰화
train_input_ids, _, _ = sp_tokenize(sp, train_processed['document_morphs'])
train_attention_masks = (train_input_ids != PAD_IDX).long()
train_labels_tensor = torch.tensor(train_processed['label'].values, dtype=torch.float)

# 테스트 데이터 토큰화
test_input_ids, _, _ = sp_tokenize(sp, test_processed['document_morphs'])
test_attention_masks = (test_input_ids != PAD_IDX).long()
test_labels_tensor = torch.tensor(test_processed['label'].values, dtype=torch.float)

print(f"✅ 훈련 데이터 토큰화 및 텐서 변환 완료. (Shape: {train_input_ids.shape})")
print(f"✅ 테스트 데이터 토큰화 및 텐서 변환 완료. (Shape: {test_input_ids.shape})")
# ⭐⭐⭐ [END NEW] ⭐⭐⭐


# ================================================================================
# ✅ 셀 15: DataLoader 생성 (기존 코드 - 변경 없음)
# ================================================================================
from torch.utils.data import TensorDataset, DataLoader, random_split

def create_dataloaders(train_input_ids, train_attention_masks, train_labels,
                       test_input_ids, test_attention_masks, test_labels,
                       batch_size=32, val_ratio=0.2):
    """
    Train/Val/Test DataLoader 생성

    Args:
        train_input_ids: 훈련 데이터 입력 ID 텐서
        train_attention_masks: 훈련 데이터 어텐션 마스크 텐서
        train_labels: 훈련 레이블
        test_input_ids: 테스트 데이터 입력 ID 텐서
        test_attention_masks: 테스트 데이터 어텐션 마스크 텐서
        test_labels: 테스트 레이블
        batch_size: 배치 크기
        val_ratio: validation 비율

    Returns:
        train_loader, val_loader, test_loader
    """
    # Train 데이터에서 train/val split
    train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
    val_size = int(len(train_dataset) * val_ratio)
    train_size = len(train_dataset) - val_size
    train_split, val_split = random_split(train_dataset, [train_size, val_size])

    # DataLoader 생성
    train_loader = DataLoader(train_split, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_split, batch_size=batch_size, shuffle=False)

    # Test DataLoader
    test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"✅ DataLoader 생성 완료!")
    print(f"   Train samples: {train_size:,} → {len(train_loader)} batches")
    print(f"   Val samples: {val_size:,} → {len(val_loader)} batches")
    print(f"   Test samples: {len(test_dataset):,} → {len(test_loader)} batches")
    print(f"   Batch size: {batch_size}")

    return train_loader, val_loader, test_loader


# ===== 사용 예시 =====
# 전처리 및 토큰화 단계에서 생성된 변수들을 사용합니다.
# train_input_ids, train_attention_masks, train_labels_tensor
# test_input_ids, test_attention_masks, test_labels_tensor

train_loader, val_loader, test_loader = create_dataloaders(
    train_input_ids,      # 훈련 데이터 입력 ID
    train_attention_masks, # 훈련 데이터 어텐션 마스크
    train_labels_tensor,  # 훈련 레이블
    test_input_ids,       # 테스트 데이터 입력 ID
    test_attention_masks, # 테스트 데이터 어텐션 마스크
    test_labels_tensor,   # 테스트 레이블
    batch_size=32,
    val_ratio=0.2
)

# 확인
print("\n" + "=" * 60)
print("📊 DataLoader 샘플 확인")
print("=" * 60)

for batch in train_loader:
    x, mask, y = batch # input_ids, attention_mask, label
    print(f"Train - 입력 텐서 shape: {x.shape}")
    print(f"Train - 어텐션 마스크 shape: {mask.shape}")
    print(f"Train - 레이블 shape: {y.shape}")
    break

for batch in val_loader:
    x, mask, y = batch # input_ids, attention_mask, label
    print(f"Val - 입력 텐서 shape: {x.shape}")
    print(f"Val - 어텐션 마스크 shape: {mask.shape}")
    print(f"Val - 레이블 shape: {y.shape}")
    break

for batch in test_loader:
    x, mask, y = batch # input_ids, attention_mask, label
    print(f"Test - 입력 텐서 shape: {x.shape}")
    print(f"Test - 어텐션 마스크 shape: {mask.shape}")
    print(f"Test - 레이블 shape: {y.shape}")
    break

# ================================================================================
# ✅ 셀 17: 하이퍼파라미터 정의 (기존 코드 - 변경 없음)
# ================================================================================
import torch.nn as nn
import torch.nn.functional as F

# 공통 하이퍼파라미터 정의
VOCAB_SIZE = 8000                  # 단어 사전의 크기
EMBEDDING_DIM = 100                # 임베딩 벡터의 차원
HIDDEN_DIM = 128                   # LSTM의 은닉 상태 차원
OUTPUT_DIM = 1                     # 출력 차원 (긍정=1, 부정=0 -> 1개)
N_LAYERS = 2                       # LSTM 레이어 개수
BIDIRECTIONAL = True               # 양방향 RNN/LSTM 여부
DROPOUT_RATE = 0.2                 # 드롭아웃 비율
PAD_IDX = 0                        # 패딩 인덱스 (0)


# ================================================================================
# ✅ 셀 18: LSTM 모델 정의 (기존 코드 - 변경 없음)
# ================================================================================
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()

        # 1. 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # 2. LSTM 레이어
        self.lstm = nn.LSTM(input_size=embedding_dim,
                           hidden_size=hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           batch_first=True,
                           dropout=dropout)

        # 3. FC 레이어
        fc_input_dim = hidden_dim * 2
        self.fc = nn.Linear(fc_input_dim, output_dim)

        # 4. 드롭아웃
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        embedded = self.embedding(text)

        # 2. LSTM
        _output, (hidden, cell) = self.lstm(embedded)

        # 3. 마지막 레이어의 은닉 상태 결합 (양방향 처리)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        # 4. FC 레이어 통과
        prediction = self.fc(hidden)

        return prediction.squeeze(1)


# ================================================================================
# ✅ 셀 19: 훈련/평가 함수 정의 (기존 코드 - 변경 없음)
# ================================================================================
import torch
import torch.optim as optim
import time

# 0. GPU 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. 헬퍼 함수 정의
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 2. 훈련 함수 정의
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for texts, masks, labels in iterator: # Unpack 3 values: input_ids, attention_mask, labels
        texts = texts.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels.float())
        acc = binary_accuracy(predictions, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # 클리핑
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# 3. 평가 함수 정의
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for texts, masks, labels in iterator: # Unpack 3 values: input_ids, attention_mask, labels
            texts = texts.to(device)
            labels = labels.to(device)
            predictions = model(texts)
            loss = criterion(predictions, labels.float())
            acc = binary_accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


# ================================================================================
# ✅ 셀 20: 모델 훈련 (기존 코드 - 변경 없음)
# ================================================================================
# 모델 설정
lstm_model = LSTMModel(
    VOCAB_SIZE,
    EMBEDDING_DIM,
    HIDDEN_DIM,
    OUTPUT_DIM,
    N_LAYERS,
    BIDIRECTIONAL,
    DROPOUT_RATE,
    PAD_IDX
).to(device)

save_path = 'best_model_lstm.pt'
N_EPOCHS = 20
patience = 5
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, lstm_model.parameters()), lr=0.0001)

# Early stopping 변수
best_valid_loss = float('inf')
patience_counter = 0

train_losses, train_accs, valid_losses, valid_accs = [], [], [], []

print(f"\n{'='*60}")
print(f"--- LSTM Model Training starts ---")
print(f"{'='*60}\n")

# 학습 루프
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(lstm_model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(lstm_model, val_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # 기록
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)

    # Early Stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), save_path)
        patience_counter = 0
        print(f'\t>> Validation loss improved ({best_valid_loss:.3f}). Model saved.')
    else:
        patience_counter += 1
        print(f'\t>> Validation loss did not improve. Counter: {patience_counter}/{patience}')
        if patience_counter >= patience:
            print(f'--- Early stopping triggered after {epoch+1} epochs ---')
            break

# 테스트 평가
print(f"\n--- Loading best LSTM model for test evaluation ---")
lstm_model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate(lstm_model, test_loader, criterion)

print(f"\n--- LSTM Model Test Results (Best Model) ---")
print(f'\tTest Loss: {test_loss:.3f}')
print(f'\tTest Acc:  {test_acc*100:.2f}%')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
None
샘플 텍스트 전처리 결과

[1] 원본: 어제 본 영화 진짜 재밌었음!!! 또 보고 싶어 😂
    결과: 어제 본 영화 진짜 재밌었음 ! 또 보고 싶어

[2] 원본: 나는 오늘 아침에 학교에 갔다. 근데 너무 졸렸음ㅋㅋㅋㅋ
    결과: 나는 오늘 아침 학교 갔다 . 근데 너무 졸렸음

[3] 원본: 밥은 먹었니?? 아직이야... 점심에 같이 먹자!!!
    결과: 밥은 먹었니 ? 아직이야 . 점심 같이 먹자 !

[4] 원본: 메캅 형태소 분석은 한국어 처리에서 많이 사용돼 👍
    결과: 메캅 형태소 분석 한국어 처리에서 많이 사용돼

[5] 원본: 파이썬으로 토큰 빈도와 품사 분포를 시각화해 보자!!!
    결과: 파이썬 토큰 빈도 품사 분포 시각화해 보자 !

[6] 원본: 요즘 코사인 유사도 기반 벡터 검색으로 RG 구축을 많이 해!!
    결과: 요즘 코사인 유사 기반 벡터 검색 rg 구축을 많이 해 !

[7] 원본: 에이전트는 외부 도구를 호출해 작업을 자동화할 수 있어. 굳!
    결과: 에이전트 외부 도구 호출해 작업을 자동화할 수 있어 . 굳 !

[8] 원본: 이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ
    결과: 영화 진짜 미쳤다 ! 너무 재밌음

[9] 원본: 배우 연기 굳이 훌륭했음,